In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import random
import os.path
from humanize import naturalsize
%load_ext google.cloud.bigquery

In [2]:
#Same credential hooplah as before

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/jackc/OneDrive/Documents/MSBA/Applied Data Analytics/Wedge/cloughwedge2021-3249989ff592.json"
client = bigquery.Client(project = 'cloughwedge2021')
gbq_proj_id = "cloughwedge2021"

In [3]:
#Write a query into a variable that will only take a list of distinct owner ID's not including owner ID's of 3.
#This is because we want to exclude non-owners.
query = '''SELECT DISTINCT(Card_No)
FROM `cloughwedge2021.cloughwedge2021.transArchive*`
WHERE Card_No!=3.0'''

In [4]:
owners = client.query(query,location="US")
#Just to see the results:
#rows = owners.result()

#for row in rows:
#    print(row[0])

In [5]:
#Create a list of owners. This will iterate over the query results from above.
owner_list = []
for owner in owners:
    owner_list.append(owner[0])

In [6]:
len(owner_list)

27207

In [7]:
owner_list[0:10]

[48312.0,
 48424.0,
 48308.0,
 48440.0,
 48455.0,
 48311.0,
 48324.0,
 50021.0,
 23170.0,
 12320.0]

In [8]:
#Here is where we create our random sample of owners. After testing the sample size in the last line of code,
#changing the owner size to 430 gave us a reasonably sized file of about 258 MB.

owner_sample = random.sample(owner_list,430)

In [9]:
#Start of the query for creating our file of transactions per owner. This is not yet a complete query, we're going to add 
#to it in the following lines. Essentially, the goal here is to create a loop that iterates over each ownerID in the
#owner sample and add it to the query. The IN statement is a way of testing if an expression matches a value. In this case,
#the IN command will be bringing in all transaction records with owner ID's that are listed. Since we're looping over the
#owner list and adding every owner ID, this will bring in every owner on the list.

owner_transactions = '''SELECT * 
FROM `cloughwedge2021.cloughwedge2021.transArchive*`
WHERE Card_No IN('''

In [10]:
#Iterate over the owner sample list, which does not contain non-owners, and add their owner number to the query.

for owner in owner_sample:
    owner_transactions += str(owner)+','

In [11]:
#Finish writing the query by adding a parenthesis after the last character to end the IN statement

owner_transactions = owner_transactions[:-1]+')'

In [12]:
random_transactions = client.query(owner_transactions,location="US")

In [13]:
#This is just one record and I did this to see what the query was bringing in. Seems like it's bringing in the data
#correctly
for transactions in random_transactions:
    print('\t'.join([str(x) for x in transactions]))
    break

2016-06-04 13:25:59+00:00	6.0	97.0	75.0	0007468210761	O.Grapefruit Juice 32oz Knu	I	 	 	1.0	1.0	0.0	2.853	3.39	3.39	4.39	None	0.0	0.0	1.0	0.0	1.0	0.0	1.0	1.0	0.0	10.0	1.0	0.0	0.0	0.0	0.0	0.0	False	True	0.0	0.0	0.0	None	None	True	0.0	1.0	True	0.0	13364.0	1.0	0.0	0.0	45.0


In [14]:
headers = ['DateTime', 'Register_Num','Emp_Num',"Trans_Num", "UPC","Description",'Trans_Type',"Trans_SubType",'Trans_Status',
           'Department','Quantity','Scale','Cost','Unit_Price','Total','Reg_Price','Alt_Price','Tax',
           "TaxExempt",'Foodstamp','WicAble','Discount','memDiscount','Discountable','DiscountType',
           'Voided','PercentDiscount','ItemQtty','volDiscType','Volume','VolSpecial','mixMatch','Matched',
           'MemType','Staff','NumFlag','itemStatus','TenderStatus','CharFlag','varflag','BatchHeaderID','Local',
           'Organic','Display','Receipt','Card_No','Store','Branch','Match_ID','Trans_ID']

In [17]:
#Write the data to a file. I thought it would be nice to include an error trap to see if the file already existed.

if os.path.exists("sample_of_owners.txt"):
    print('this file already exists!')
else:
    with open("sample_of_owners.txt", 'w', encoding = "utf-8") as outfile:
        outfile.write(",".join(headers) + "\n")
        for row in random_transactions:
            outfile.write(",".join([str(x) for x in row])+ '\n')

In [18]:
size = os.stat('sample_of_owners.txt').st_size

print(naturalsize(size))

258.7 MB
